In [9]:
import os
import pandas as pd
import numpy as np

In [32]:
def check_nan(table, rate = 1):
    accepted_col = []
    for col in table.columns:
        na_percent = pd.isna(table[col]).sum() /  len(table)
        na_percent_risk = pd.isna(table[table.entid.isin(train_id)][col]).sum() / len(train_id) / rate
        risk_0 = pd.isna(table[table.entid.isin(train_id_0)][col]).sum() / len(train_id_0) / rate
        risk_1 =  pd.isna(table[table.entid.isin(train_id_1)][col]).sum() / len(train_id_1) / rate
        risk_2 =  pd.isna(table[table.entid.isin(train_id_2)][col]).sum() / len(train_id_2) / rate
        risk_3 = pd.isna(table[table.entid.isin(train_id_3)][col]).sum() / len(train_id_3) / rate
        if na_percent <= 0.9:
            accepted_col.append(col)
        print("col_{}, na_percent{:.2f}, na_percent_risk{:.2f}, [{:.2f},{:.2f},{:.2f},{:.2f}]".format(col,na_percent,na_percent_risk,risk_0,risk_1,risk_2,risk_3))
    print(accepted_col)
    return accepted_col

In [245]:
def fillna_year(table, use_cols): 
    new_cols = {}
    for col in use_cols:
        new_cols[col] = []
        
    table_mean = table.mean()
    table_group_mean = table.groupby("entid").apply(lambda x: x.mean())
    for i in table.index:
        row = table.loc[i]
        for col in use_cols :
            if np.isnan(row[col]):
                mean_row = table_group_mean[table_group_mean["entid"] == row.entid]
                if len(mean_row) == 1 and ~np.isnan(mean_row[col].values[0]):
                    new_cols[col].append(mean_row[col].values[0])
                else:
                    new_cols[col].append(table_mean[col])
            else:
                new_cols[col].append(row[col])  
    return new_cols

In [33]:
dir_ = "./trainning_data/corporate_attributes/"
files = os.listdir(dir_)
print(files)

['company_ar_socialfee.csv', 'company_modify.csv', '.ipynb_checkpoints', 'company_ar_alterstockinfo.csv', 'el_company_history_manager.csv', 'REGCAPCUR.xls', 'company_base_info.csv', 'company_ar_assetsinfo.csv', 'company_ar.csv', 'tax_company.csv', 'tax_qianshui.csv', 'tax_abnormal.csv', 'el_company_history_inv.csv', 'company_ar_nz.csv', 'tax_year.csv']


In [34]:
for file in files:
    if file.endswith("csv"):
        csv_temp = pd.read_csv(dir_ + file)
        print(file, len(csv_temp))

company_ar_socialfee.csv 10396
company_modify.csv 77622
company_ar_alterstockinfo.csv 3679
el_company_history_manager.csv 11296
company_base_info.csv 12279
company_ar_assetsinfo.csv 4884
company_ar.csv 50048
tax_company.csv 24864
tax_qianshui.csv 290
tax_abnormal.csv 359
el_company_history_inv.csv 10170
company_ar_nz.csv 1365
tax_year.csv 30346


In [35]:
train = pd.read_csv("./trainning_data/label_risk_company.csv")
train_id = set(train.entid)

train_id_0 = set(train[train["CaseType"] == 0].entid)
train_id_1 = set(train[train["CaseType"] == 1].entid)
train_id_2 = set(train[train["CaseType"] == 2].entid)
train_id_3 = set(train[train["CaseType"] == 3].entid)

In [36]:
# company_base_info.csv 

In [37]:
base_info = pd.read_csv("./trainning_data/corporate_attributes/company_base_info.csv")

In [90]:
use_entid = pd.DataFrame(base_info.entid.unique())

In [38]:
base_info_cols = check_nan(base_info)

col_entid, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_honor_num, na_percent1.00, na_percent_risk1.00, [1.00,1.00,0.98,1.00]
col_illegad_num, na_percent1.00, na_percent_risk1.00, [1.00,1.00,0.99,1.00]
col_yea_sub, na_percent0.96, na_percent_risk1.00, [1.00,0.98,1.00,1.00]
col_web_dum, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ENTTYPE, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_INDUSTRYPHY, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_REGCAP, na_percent0.04, na_percent_risk0.03, [0.00,0.16,0.06,0.03]
col_REGCAPCUR, na_percent0.07, na_percent_risk0.10, [0.06,0.12,0.04,0.10]
col_RECCAP, na_percent0.23, na_percent_risk0.14, [0.02,0.20,0.16,0.15]
col_PARNUM, na_percent0.90, na_percent_risk0.88, [0.96,0.92,0.91,0.87]
col_LIMPARNUM, na_percent0.90, na_percent_risk0.88, [0.96,0.88,0.89,0.87]
col_PARFORM, na_percent0.91, na_percent_risk0.91, [0.98,0.86,0.88,0.91]
col_EXENUM, na_percent0.81, na_percent_risk0.63, [0.53,

In [39]:
base_info_features = base_info[base_info_cols]

In [ ]:
# categerial: web_dum, ENTTYPE, INDUSTRYPHY, REGCAPCUR, tax_dum
# fill_average: REGCAP, RECCAP 
base_info_features["REGCAP"] = base_info_features["REGCAP"].fillna(base_info_features["REGCAP"].mean())
base_info_features["RECCAP"] = base_info_features["RECCAP"].fillna(base_info_features["RECCAP"].mean())
# fill_unknown
base_info_features["REGCAPCUR"] = base_info_features["REGCAPCUR"].fillna("unknown")
# fill_zero: PARNUM, LIMPARNUM,certificate_num
base_info_features["PARNUM"] = base_info_features["PARNUM"].fillna(0)
base_info_features["LIMPARNUM"] = base_info_features["LIMPARNUM"].fillna(0)
base_info_features["EXENUM"] = base_info_features["LIMPARNUM"].fillna(0)
base_info_features["certificate_num"] = base_info_features["certificate_num"].fillna(0)

In [49]:
base_info_cols = check_nan(base_info_features)

col_entid, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_web_dum, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ENTTYPE, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_INDUSTRYPHY, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_REGCAP, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_REGCAPCUR, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_RECCAP, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_PARNUM, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_LIMPARNUM, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_EXENUM, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_cancel_dum, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_certificate_num, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_tax_dum, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
['entid', 'web_dum', 'ENTTYPE', 'INDUSTRYPHY', 'REG

In [239]:
base_info_features.to_csv("./trainning_data/corporate_attributes/company_base_info_new.csv")

In [31]:
#统一categrical，数值类变换， 分箱子（离散化）进行预测。

entid                int64
web_dum              int64
ENTTYPE              int64
INDUSTRYPHY          int64
REGCAP             float64
REGCAPCUR          float64
RECCAP             float64
PARNUM             float64
LIMPARNUM          float64
EXENUM             float64
cancel_dum           int64
certificate_num    float64
tax_dum              int64
dtype: object

In [50]:
# company_ar
# categorical: BUSST
# todo 
# categorical：员工数量分箱
# 员工数量是否出现减少

In [51]:
company_ar = pd.read_csv("./trainning_data/corporate_attributes/company_ar.csv")

In [64]:
company_ar["EMPNUM"] = company_ar["EMPNUM"].fillna(0)
company_ar["WOMEMPNUM"] = company_ar["WOMEMPNUM"].fillna(0)

In [ ]:
# company_ar_socialfee.csv

In [240]:
company_ar_assetsinfo = pd.read_csv("./trainning_data/corporate_attributes/company_ar_assetsinfo.csv")

In [242]:
company_ar_assetsinfo_cols = check_nan(company_ar_assetsinfo)

col_entid, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ANCHEYEAR, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ispublic, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ASSGRO, na_percent0.09, na_percent_risk0.04, [0.00,0.00,0.12,0.03]
col_LIAGRO, na_percent0.11, na_percent_risk0.04, [0.00,0.00,0.12,0.04]
col_VENDINC, na_percent0.09, na_percent_risk0.02, [0.00,0.00,0.07,0.01]
col_MAIBUSINC, na_percent0.10, na_percent_risk0.02, [0.00,0.00,0.09,0.01]
col_PROGRO, na_percent0.14, na_percent_risk0.05, [0.00,0.00,0.13,0.04]
col_NETINC, na_percent0.08, na_percent_risk0.01, [0.00,0.00,0.01,0.01]
col_RATGRO, na_percent0.05, na_percent_risk0.01, [0.00,0.00,0.01,0.01]
col_TOTEQU, na_percent0.13, na_percent_risk0.04, [0.00,0.00,0.13,0.04]
['entid', 'ANCHEYEAR', 'ispublic', 'ASSGRO', 'LIAGRO', 'VENDINC', 'MAIBUSINC', 'PROGRO', 'NETINC', 'RATGRO', 'TOTEQU']


In [246]:
new_company_ar_assetsinfo = fillna_year(company_ar_assetsinfo, company_ar_assetsinfo_cols)

In [249]:
new_company_ar_assetsinfo = pd.DataFrame(new_company_ar_assetsinfo)

In [252]:
new_company_ar_assetsinfo.entid = new_company_ar_assetsinfo.entid.astype(int)

In [253]:
new_company_ar_assetsinfo.to_csv("./trainning_data/revised_attribute/company_ar_assetsinfo_new.csv")

In [ ]:
#tax_info

In [140]:
tax_year = pd.read_csv("./trainning_data/corporate_attributes/tax_year.csv")
tax_year_gourp_mean = tax_year.groupby("entid").apply(lambda x: x.mean())

In [177]:
tax_year_mean = tax_year.mean()

In [179]:
tax_year_mean["profit_loss"]

0.3693973872066549

In [87]:
tax_year_cols = check_nan(tax_year)

col_entid, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_year, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_islatest, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_profit_loss, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_revenue, na_percent0.10, na_percent_risk0.35, [0.09,0.26,0.45,0.35]
col_sales_cost, na_percent0.17, na_percent_risk0.41, [0.21,0.66,0.95,0.38]
col_sales_tax, na_percent0.18, na_percent_risk0.38, [0.13,0.44,0.68,0.37]
col_sales_expense, na_percent0.24, na_percent_risk0.42, [0.21,0.54,1.16,0.39]
col_G_expense, na_percent0.10, na_percent_risk0.17, [0.04,0.32,0.63,0.15]
col_finance_expense, na_percent0.15, na_percent_risk0.25, [0.09,0.32,0.62,0.24]
col_asset_devaluation_losses, na_percent0.36, na_percent_risk0.48, [0.23,1.04,1.49,0.42]
col_fair_value_change, na_percent0.36, na_percent_risk0.49, [0.23,1.04,1.66,0.43]
col_investment_income, na_percent0.36, na_percent_risk0.46, [0.23,0.98,1.34,0.41]
col_op

In [227]:
new_cols = {}
for tax_year_col in tax_year_cols:
    new_cols[tax_year_col] = []

In [228]:
for i in tax_year.index:
    row = tax_year.loc[i]
    for tax_year_col in tax_year_cols :
        if np.isnan(row[tax_year_col]):
            mean_row = tax_year_gourp_mean[tax_year_gourp_mean["entid"] == row.entid]
            if len(mean_row) == 1 and ~np.isnan(mean_row[tax_year_col].values[0]):
                new_cols[tax_year_col].append(mean_row[tax_year_col].values[0])
            else:
                new_cols[tax_year_col].append(tax_year_mean[tax_year_col])
        else:
            new_cols[tax_year_col].append(row[tax_year_col])  

In [110]:
tax_year_gourp_mean = tax_year.groupby("entid").apply(lambda x: x.mean())

In [134]:
len(tax_year_gourp_mean[tax_year_gourp_mean["entid"] == 1109606]["revenue"])

0

In [135]:
tax_year = tax_year.groupby("entid").count()["year"] > 2

In [234]:
new_tax_year = pd.DataFrame(new_cols)

In [235]:
new_tax_year.entid = new_tax_year.entid.astype(int)

In [237]:
new_tax_year.to_csv("./trainning_data/corporate_attributes/tax_year_new.csv")

In [254]:
# company_ar_socialfee.csv

In [259]:
social_fee = pd.read_csv("./trainning_data/corporate_attributes/company_ar_socialfee.csv")

In [260]:
social_fee_usecol = check_nan(social_fee)

col_entid, na_percent0.00, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ispublic, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_ANCHEYEAR, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_so1, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_so2, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_so3, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_so4, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_so5, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_totalWagesSo1, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_totalWagesSo2, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_totalWagesSo3, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_totalWagesSo5, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_totalPaymentSo1, na_percent0.96, na_percent_risk0.00, [0.00,0.00,0.00,0.00]
col_totalPaymentSo2, na_percent0.96, na_percent_ris

In [261]:
len(social_fee)

10396